In [ ]:
!pip install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from heapq import nlargest
from string import punctuation
from rouge import Rouge
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
text1 = """
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A include fever, tiredness, loss of appetite, nausea and abdominal discomfort. Fargo Catholic Diocese in North Dakota (pictured) is where the bishop is located .
"""
refSumm1 = """
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
"""


text2 = """
A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titleyâ€™s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Rachel Titley, 28 (right). She died later from her injuries . The head-on crash took place in October 2013. Mr Eccleston-Todd's car was barely recognisable (pictured) Police said Eccleston-Todd had drunk at least three or four pints of beer before getting behind the wheel. He was found guilty of causing death by dangerous driving at Portsmouth Crown Court yesterday. Miss Titley, a 28-year-old solicitorâ€™s clerk from Cowes, Isle of Wight, had also spent the evening with friends at a pub but had not drunk any alcohol, police said. She was driving responsibly and there was â€˜nothing she could have done to avoid the collisionâ€™, they added. Lindsay Pennell, prosecuting, said: â€˜Craig Eccleston-Toddâ€™s driving resulted in the tragic death of a young woman, Rachel Titley, a death that could have been avoided. â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the path of Miss Titleyâ€™s oncoming car. Miss Titley was pulled the wreckage of herÂ Daihatsu Cuore but died later from her injuries in hospital . â€˜Miss Titley [had] a bright future ahead of her. She was also returning home having spent an enjoyable evening with friends and was driving responsibly. â€˜She had arranged to contact her friends when she got home to confirm that she had arrived safely. Her friends sadly never heard from her after they parted company. â€˜Miss Titleyâ€™s death in these circumstances reiterates the danger of using a hand-held mobile phone whilst driving.â€™ Police were unable to take breath or blood tests from Eccleston-Todd immediately, but in tests several hours after the accident he was only marginally under the drink-drive limit. The judge agreed with police that he would have been over the limit at the time his red Citroen hit Miss Titleyâ€™s blue Daihatsu Cuore on a road near Yarmouth, Isle of Wight, on October 11, 2013. His phone records showed he was also texting around the time of the crash. PC Mark Furse, from Hampshire constabularyâ€™s serious collision investigation unit, said: 'Our thoughts are with Rachel's family at this time. She had been out with friends at a pub in Shalfleet that evening, but had not had any alcohol. 'Our investigation showed that there was nothing she could have done to avoid the collision and sadly it cost her her life. 'Mr Eccleston-Todd had left work in Yarmouth and met with friends at a pub where he drank at least three to four pints of lager. He hadn't long left the pub to return home when the collision occurred at around 9.30pm. 'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in summing up today, the judge agreed. 'The analysis of his phone records showed that he was texting on his phone around the time of the collision so it's highly likely this would also have contributed to his dangerous driving and loss of control.' Eccleston-Todd was found guilty of causing death by dangerous driving following a trial at Portsmouth Crown Court (pictured) He added: 'Mr Eccleston-Todd will now spend six years behind bars, but Rachel's family have lost her forever. 'I hope this will make people think twice before drinking any alcohol and getting behind the wheel, or using a phone once they're on the road. 'The dangers of drink driving and driving whilst using a mobile phone are obvious. Those who continue to do so risk spending a substantial time in prison. This case highlights just how tragic the consequences of committing these offences can be.' â€˜Mr Eccleston-Todd will now spend six years behind bars, but Rachelâ€™s family have lost her for ever. I hope this will make people think twice before drinking any alcohol and getting behind the wheel, or using a phone once theyâ€™re on the road. This case highlights just how tragic the consequences of committing these offences can be.â€™ Eccleston-Todd, of Newport, Isle of Wight, was also disqualified from driving for eight yearsÂ after which he will have to complete an extended re-test.
"""
refSumm2 = """
Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .
Was using phone when he veered across road in Yarmouth, Isle of Wight .
Crashed head-on into 28-year-old Rachel Titley's car, who died in hospital .
Police say he would have been over legal drink-drive limit at time of crash .
He was found guilty at Portsmouth Crown Court of causing death by dangerous driving .
"""

text3 = """
Fleetwood are the only team still to have a 100% record in Sky Bet League One as a 2-0 win over Scunthorpe sent Graham Alexanderâ€™s men top of the table. The Cod Army are playing in the third tier for the first time in their history after six promotions in nine years and their remarkable ascent shows no sign of slowing with Jamie Proctor and Gareth Evans scoring the goals at Glanford Park. Fleetwood were one of five teams to have won two out of two but the other four clubs - Peterborough, Bristol City, Chesterfield and Crawley - all hit their first stumbling blocks. Posh were defeated 2-1 by Sheffield United, who had lost both of their opening contests. Jose Baxterâ€™s opener gave the Blades a first-half lead, and although it was later cancelled out by Shaun Brisleyâ€™s goal, Ben Davies snatched a winner six minutes from time. In the lead: Jose Baxter (right) celebrates opening the scoring for Sheffield United . Up for the battle: Sheffield United's Michael Doyle (left) challenges Peterborough's Kyle Vassell in a keenly-contested clash . Bristol City, who beat Nigel Cloughâ€™s men on the opening day, were held to a goalless draw by last season's play-off finalists Leyton Orient while Chesterfield, the League Two champions, were beaten 1-0 by MK Dons, who play Manchester United in the Capital One Cup in seven daysâ€™ time. Arsenal loanee Benik Afobe scored the only goal of the game just after the break. Meanwhile, Crawley lost their unbeaten status, while Bradford maintained theirs, thanks to a 3-1 win for the Bantams. James Hanson became the first player to score against Crawley this season after 49 minutes before Joe Walsh equalised five minutes later. Heads up: Bristol City's Korey Smith (left) and Leyton Orient's Lloyd James go up for a header . But strikes from Billy Knott and Mason Bennett sealed an impressive away win Phil Parkinson's men. Bradford are now second behind Fleetwood after Doncasterâ€™s stoppage-time equaliser meant Preston, for whom Joe Garner signed a new contract earlier on Tuesday, were held to a 1-1 draw which slipped them down the table. Chris Humphrey looked to have secured the points for the Lilywhites but Nathan Tyson struck a last-gasp leveller. Stand-in striker Matt Done scored a hat-trick for Rochdale in the eveningâ€™s high-scoring affair as Crewe were hammered 5-2. Marcus Haber marked his full Railwaymen debut with a brace but Doneâ€™s treble and goals from Ian Henderson and Peter Vincenti helped Keith Hillâ€™s men to a big away victory. There were plenty of goals between Coventry and Barnsley too in a 2-2 draw with all four goals coming in the first half. Josh McQuoid and Jordan Clarke twice gave the Sky Blues the lead, but the Tykes earned a point thanks to strikes from Conor Hourihane and Leroy Lita. Notts County recorded a 2-1 home win over Colchester with Ronan Murray and Liam Noble on target. Freddie Sears replied for Colchester. James Wilson's second half equaliser earned Oldham a points against Port Vale after Tom Pope's opener and Yeovil claimed a 2-1 away victory at Walsall with Kevin Dawson striking a late winner. Tom Bradshaw had equalised after veteran James Hayter gave the Glovers the lead. Finally, Swindon held Gillingham to a 2-2 draw thanks to Stephen Bywaterâ€™s last-minute own goal. Danny Kedwell and Kortney Hause twice gave the Gills the lead but Andy Williams pulled Swindon level before Bywater dropped Raphael Branco's cross into his own net.
"""
refSumm3 = """
Fleetwood top of League One after 2-0 win at Scunthorpe .
Peterborough, Bristol City, Chesterfield and Crawley all drop first points of the season .
Stand-in striker Matt Done scores a hat-trick as Rochdale thrash Crewe 5-2 .
Wins for Notts County and Yeovil .
Coventry/Bradford and Oldham/Port Vale both end in draws .
A late Stephen Bywater own goal denies Gillingham three points against Millwall .
"""

article = []
refehighlights = []
originalLen = []
newLenTfiDF = []
newLenBOW = []

# Sample array to hold all the article
article.append(text1)
article.append(text2)
article.append(text3)

# Sample array to hold the original length of the article
originalLen.append(len(text1))
originalLen.append(len(text2))
originalLen.append(len(text3))


# Sample reference array to hold all the heighlights 
refehighlights.append(refSumm1)
refehighlights.append(refSumm1)
refehighlights.append(refSumm1)

# Sample array to hold the generated array from TFIDF
generatedArrayTFIDF = []
# Sample array to hold the generated array from BOW
generatedArrayBOW = []


In [ ]:
stopwords = list(STOP_WORDS)
punctuation = punctuation + '\n'

print(stopwords)
print(punctuation)

['get', 'either', 'during', 'once', 'becoming', 'whereafter', 'wherever', 'two', 'whereby', '‘d', 'nevertheless', 'least', 'go', 're', 'amount', 'in', 'further', 'among', 'am', 'hundred', 'each', 'should', 'nobody', 'together', 'forty', 'name', '‘re', 'not', 'what', 'there', 'made', 'many', 'itself', 'formerly', 'off', 'move', 'whenever', 'of', 'that', 'front', '‘ve', 'cannot', 'hers', 'then', 'own', 'put', 'beyond', 'no', 'via', 'n‘t', 'others', 'whom', 'one', 'see', 'indeed', 'therefore', 'eight', 'become', 'side', 'almost', 'without', 'did', 'nine', 'and', 'may', 'against', 'anything', 'thereafter', 'which', 'up', 'became', 'thereupon', "n't", 'she', 'twenty', 'until', 'him', 'around', 'any', 'whose', 'six', 'by', 'yourselves', 'i', 'ours', 'fifty', '’ve', 'fifteen', 'again', 'everywhere', 'when', 'namely', 'used', 'something', 'nor', 'about', 'afterwards', 'amongst', '’d', 'who', 'four', 'now', 'same', 'using', 'twelve', 'latterly', 'else', 'all', 'everything', 'thus', 'always', 'w

TF-IDF

In [ ]:
# TF-IDF Summary
# Clearing the array everytime it runs
generatedArrayTFIDF = []
newLenTfiDF = []
i = 0
while(i<len(article)):
  sentences = article[i].split(".")
  # Create a TfidfVectorizer
  vectorizer = TfidfVectorizer()

  # Transform the sentences into a TF-IDF representation
  tfidf = vectorizer.fit_transform(sentences)
  feature_names = vectorizer.get_feature_names()

  # Find the n most important sentences
  n = 2
  indices = np.argsort(tfidf.toarray().sum(axis=1))[-n:]
  most_important_sentences = [sentences[i] for i in indices]

  # Generate the text summary
  summary = " ".join(most_important_sentences) + "."
  generatedArrayTFIDF.append(summary)
  newLenTfiDF.append(len(summary))
  print(summary)
  print("------------------------------------------------------------------------------")

  i+=1

 Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A   The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.
------------------------------------------------------------------------------
 'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in summing up today, the judge agreed  â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the pa

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
generatedArrayTFIDF

[' Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A   The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.',
 " 'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in summing up today, the judge agreed  â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the path of Miss Titleyâ€™s oncoming car.",
 " The Cod Army are playing in the 

In [ ]:
# Length of Original Text and Summary:
i = 0
while i<len(article):
  print("Original Text: ",originalLen[i],", Summary: ",newLenTfiDF[i])
  i+=1

Original Text:  1213 , Summary:  389
Original Text:  4785 , Summary:  566
Original Text:  3461 , Summary:  529


Rouge for TFIDF

In [ ]:
rouge = Rouge()

print("Rouge scores for TFIDF")
print()
print()

i = 0
while(i<len(generatedArrayTFIDF)):
  generated_summ = generatedArrayTFIDF[i]
  reference_summ = refehighlights[i]

  scores = rouge.get_scores(generated_summ,reference_summ,avg = True)
  rouge_1 = scores['rouge-1']['f']
  rouge_2 = scores['rouge-2']['f']
  rouge_l = scores['rouge-l']['f']

  # Print the ROUGE scores
  print(f"ROUGE-1: {rouge_1:.2f}")
  print(f"ROUGE-2: {rouge_2:.2f}")
  print(f"ROUGE-L: {rouge_l:.2f}")

  print()
  print("---------------------------------------------------------")
  print()
  i+=1

Rouge scores for TFIDF


ROUGE-1: 0.39
ROUGE-2: 0.19
ROUGE-L: 0.39

---------------------------------------------------------

ROUGE-1: 0.14
ROUGE-2: 0.02
ROUGE-L: 0.11

---------------------------------------------------------

ROUGE-1: 0.11
ROUGE-2: 0.00
ROUGE-L: 0.09

---------------------------------------------------------



BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# Clearing the array everytime it runs
generatedArrayBOW = []
newLenBOW = []

# Create the bag-of-words model
vectorizer = CountVectorizer()


i = 0
while(i<len(article)):
  sentences = article[i].split(".")
  
  # Transform the sentences to vectors
  sentence_vectors = vectorizer.fit_transform(sentences)
  
  # Calculate the score for each sentence
  scores = sentence_vectors.sum(axis=1)

  # Normalize the scores
  normalized_scores = scores / sentence_vectors.sum(axis=1).sum()

  n = 2
  indices = np.argsort(sentence_vectors.toarray().sum(axis=1))[-n:]
  imp_sen = [sentences[k] for k in indices]

  summary = " ".join(imp_sen) + "."
  print(summary)
  generatedArrayBOW.append(summary)
  newLenBOW.append(len(summary))

  
  print("------------------------------------------------------------------------------")
  i+=1


 Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A   The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.
------------------------------------------------------------------------------
 â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the path of Miss Titleyâ€™s oncoming car  'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in 

In [ ]:
generatedArrayBOW

[' Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A   The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.',
 " â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the path of Miss Titleyâ€™s oncoming car  'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in summing up today, the judge agreed.",
 " The Cod Army are playing in the 

In [ ]:
# Length of Original Text and Summary:
i = 0
while i<len(article):
  print("Original Text: ",originalLen[i],", Summary: ",newLenBOW[i])
  i+=1

Original Text:  1213 , Summary:  389
Original Text:  4785 , Summary:  566
Original Text:  3461 , Summary:  529


Rougr for BOW

In [ ]:
rouge = Rouge()

print("Rouge scores for BOW")
print()
print()

i = 0
while(i<len(generatedArrayBOW)):
  generated_summ = generatedArrayBOW[i]
  reference_summ = refehighlights[i]

  scores = rouge.get_scores(generated_summ,reference_summ,avg = True)
  rouge_1 = scores['rouge-1']['f']
  rouge_2 = scores['rouge-2']['f']
  rouge_l = scores['rouge-l']['f']

  # Print the ROUGE scores
  print(f"ROUGE-1: {rouge_1:.2f}")
  print(f"ROUGE-2: {rouge_2:.2f}")
  print(f"ROUGE-L: {rouge_l:.2f}")

  print()
  print("---------------------------------------------------------")
  print()
  i+=1

Rouge scores for BOW


ROUGE-1: 0.39
ROUGE-2: 0.19
ROUGE-L: 0.39

---------------------------------------------------------

ROUGE-1: 0.14
ROUGE-2: 0.02
ROUGE-L: 0.13

---------------------------------------------------------

ROUGE-1: 0.11
ROUGE-2: 0.00
ROUGE-L: 0.09

---------------------------------------------------------



Neural Network Approach

In [ ]:
import tensorflow as tf

# Example input document and summary
document = "This is an example input document. It contains multiple sentences and words."
summary = "This is an example summary."

# Preprocess the input document and summary
document_tokens = document.split()
summary_tokens = summary.split()

# Create a vocabulary from the input document and summary
vocab = set(document_tokens + summary_tokens)
vocab = list(vocab)


lookUpTable = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(vocab, tf.cast(tf.range(len(vocab)),tf.int64)),
    num_oov_buckets=1)

# Define the neural network architecture
max_len = max(len(document_tokens), len(summary_tokens))
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab), 64, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(len(document_tokens), activation='softmax')
])
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Embedding(len(vocab), 64, input_length=len(document_tokens)))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.GlobalMaxPooling1D())
# model.add(tf.keras.layers.Dense(len(document_tokens), activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert the input document and summary into tensors
document_tensor = tf.constant([document_tokens])
summary_tensor = tf.constant([summary_tokens])

# Convert the tokens into one-hot encodings
ids_doc = lookUpTable.lookup(document_tensor)
ids_summ = lookUpTable.lookup(summary_tensor)
document_one_hot = tf.one_hot(ids_doc, len(vocab), dtype=tf.int32)
summary_one_hot = tf.one_hot(ids_summ, len(vocab), dtype=tf.int32)

# Train the model
model.fit(document_one_hot, summary_one_hot, epochs=10, batch_size=32)

# Evaluate the model on a test document
test_document = "This is a test document. It should be summarized correctly."
test_document_tokens = test_document.split()
test_tensor = tf.constant([test_document_tokens])
ids_test = lookUpTable.lookup(test_tensor)
test_document_one_hot = tf.one_hot(ids_test, len(vocab), dtype=tf.int32)
predicted_summary_one_hot = model.predict(test_document_one_hot)

# Convert the predicted summary back into tokens
predicted_summary_tokens = []
for i in range(len(predicted_summary_one_hot)):
    predicted_summary_tokens.append(vocab[tf.argmax(predicted_summary_one_hot[i])])

# Join the predicted summary tokens into a string
predicted_summary = " ".join(predicted_summary_tokens)

# Print the predicted summary
print(predicted_summary)

Linear Regression Approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import numpy as np

# Sample input data
documents = ['This is the first document.', 'This is the second document.', 'This is the third document.']
summaries = ['This is a summary of the first document.', 'This is a summary of the second document.', 'This is a summary of the third document.']

documents = np.array(documents)
summaries = np.array(summaries)

# Convert documents and summaries to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)
Y = vectorizer.transform(summaries)

# Split the data into training and testing sets
train_size = int(0.8 * len(documents))
X_train, Y_train = X[:train_size], Y[:train_size]
X_test, Y_test = X[train_size:], Y[train_size:]

# Train a linear regression model
reg = LinearRegression()
reg.fit(X_train, Y_train)

# Predict summaries for the testing set
Y_pred = reg.predict(X_test)

# Calculate evaluation metrics (e.g., mean squared error)
mse = mean_squared_error(Y_test, Y_pred)

print("Mean squared error: ", mse)


In [ ]:
import numpy as np

# Example input document and summary
# document = "This is an example input document. It contains multiple sentences and words."
# summary = "This is an example summary."

# Preprocess the input document and summary
# document_tokens = document.split()
# summary_tokens = summary.split()
# document_tokens = ['This is the first document.', 'This is the second document.', 'This is the third document.']
document_tokens = article
# summary_tokens = ['This is a summary of the first document.', 'This is a summary of the second document.', 'This is a summary of the third document.']
summary_tokens = refehighlights
# Convert the tokens into numpy arrays
document_array = np.array(document_tokens)
summary_array = np.array(summary_tokens)

# Create a vocabulary from the input document and summary
vocab = np.unique(np.concatenate((document_array, summary_array), axis=0))

# Create a matrix of document term frequencies
doc_term_matrix = np.zeros((len(document_tokens), len(vocab)))
for i, token in enumerate(document_tokens):
    for j, word in enumerate(vocab):
        if word == token:
            doc_term_matrix[i, j] += 1

# Create a matrix of summary term frequencies
sum_term_matrix = np.zeros((len(summary_tokens), len(vocab)))
for i, token in enumerate(summary_tokens):
    for j, word in enumerate(vocab):
        if word == token:
            sum_term_matrix[i, j] += 1

# Compute the TF-IDF scores
doc_freq = np.sum(doc_term_matrix, axis=0)
sum_freq = np.sum(sum_term_matrix, axis=0)
doc_term_freq = np.sum(doc_term_matrix > 0, axis=0)
idf = np.log((len(document_tokens)+1) / (doc_term_freq + 1)) + 1
tf_doc = np.log(doc_term_matrix + 1)
tf_sum = np.log(sum_term_matrix + 1)
tf_idf_doc = tf_doc * idf
tf_idf_sum = tf_sum * idf

# Train the linear regression model
from sklearn.linear_model import LinearRegression

X_train = tf_idf_doc
y_train = tf_idf_sum

reg = LinearRegression().fit(X_train, y_train)

# Evaluate the model on a test document
test_document = "This is a test document. It should be summarized correctly."
test_document_tokens = test_document.split()

test_doc_term_matrix = np.zeros((len(test_document_tokens), len(vocab)))
for i, token in enumerate(test_document_tokens):
    for j, word in enumerate(vocab):
        if word == token:
            test_doc_term_matrix[i, j] += 1

tf_test_doc = np.log(test_doc_term_matrix + 1)
tf_idf_test_doc = tf_test_doc * idf

predicted_tf_idf_sum = reg.predict(tf_idf_test_doc)

# Convert the predicted summary back into tokens
predicted_summary_tokens = []
for i in range(len(predicted_tf_idf_sum)):
    predicted_summary_tokens.append(vocab[np.argmax(predicted_tf_idf_sum[i])])

# Join the predicted summary tokens into a string
predicted_summary = " ".join(predicted_summary_tokens)

# Print the predicted summary
print(predicted_summary)



Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
 
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
 
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
 
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
 
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection 

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

X_train = []
Y_train = []
# samp = []
# samp.append(0)
i = 0
while(i<3):
  document = article[i]
  refSumm = refehighlights[i]

  document_tok = document.split()
  refSumm_tok = refSumm.split()

  document_array = np.array(document_tok)
  refSumm_array = np.array(refSumm_tok)

  vocab = np.unique(np.concatenate((document_array,refSumm_array), axis=0))

  # Create a matrix of document term frequencies
  doc_term_matrix = np.zeros((len(document_tok), len(vocab)))
  for i, token in enumerate(document_tok):
      for j, word in enumerate(vocab):
          if word == token:
              doc_term_matrix[i, j] += 1

  # Create a matrix of summary term frequencies
  sum_term_matrix = np.zeros((len(refSumm_tok), len(vocab)))
  for i, token in enumerate(refSumm_tok):
      for j, word in enumerate(vocab):
          if word == token:
              sum_term_matrix[i, j] += 1

  doc_freq = np.sum(doc_term_matrix,axis = 0)
  sum_freq = np.sum(sum_term_matrix,axis = 0)
  doc_term_freq = np.sum(doc_term_matrix > 0, axis=0)
  idf = np.log((len(document_tok)+1) / (doc_term_freq + 1)) + 1
  tf_doc = np.log(doc_term_matrix + 1)
  tf_sum = np.log(sum_term_matrix + 1)
  tf_idf_doc = tf_doc * idf
  tf_idf_sum = tf_sum * idf
  
  X_train.append([tf_idf_doc])
  Y_train.append(tf_idf_sum)

  i+=1

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_train = X_train.reshape((X_train.shape[0],-1))
Y_train = Y_train.reshape((Y_train.shape[0],-1))
# Fitting the Model for X_train and Y_train
reg = LinearRegression().fit(X_train,Y_train)
Test_article = """
There are a number of job descriptions waiting for Darren Fletcher when he settles in at West Brom but the one he might not have expected is Saido Berahinoâ€™s nanny. Fletcherâ€™s unveiling as the deadline day signing from Manchester United was almost eclipsed by the 21-year-old striker, who is acquiring the habit of talking himself into trouble. Ten years Berahinoâ€™s senior, Fletcher will be expected to mentor a player who told the world this week that he wanted to play for a bigger club. Tony Pulis has advised Saido Berahino to focus on his performances at West Brom . Darren Fletcher has signed for the baggies where he will be asked to provide a role model for young players . That is off the pitch. On it, the Scotland midfielder wants to prove he is good enough to cut the mustard in the Premier League after finding starts harder and harder to come by at Old Trafford. Head coach Tony Pulis believes that Fletcher, who has agreed a three-and-a-half year contract, will be captain of Albion one day. Having checked with Sir Alex Ferguson last year when he was Crystal Palace, Pulis did not need any more due diligence before moving in when a deal with West Ham collapsed. Pulis wants Fletcher to be his voice in the dressing room, especially when it comes to the younger players who may be led astray. Berahino has caught the eye with impressive performances at West Brom and suggested he could move on . Berahinoâ€™s latest outburst this week comes after he was found guilty of drink-driving and after he moodly refused to celebrate a hat-trick against Gateshead. â€˜Things are not what they used to be,â€™ added Pulis. â€˜The mentors for these young lads are just not there. These kids need guides and mentors so that the youngsters can respect them and take notice. â€˜I think Fletch will be critical to that sort of stuff but give him time to settle in. As a character, having worked with him for a week, he is first class. He got through his illness with flying colours and I see him as a future captain of the club.â€™ As for Berahino, he will escape a fine. â€˜He's been in a naughty chair. That's in my office,â€™ joked Pulis, although the underlying message was rather more serious. â€˜We've had no phone calls. He needs to stop listening to all the kerfuffle.. This is a great football club with great players. And Saido has not become that yet. Pulis praised recent recruit Darren Fletcher and feels he could be an ideal role model for Berahino . â€˜The question was whether would he like to play in a top four team and everyone wants that. His responsibility is to work for us until that happens. â€˜I've spoken to him and his people. He has to do it rather than talk about it. That's what good players do and then clubs will be interested. He's done an interview but not for what he was supposed to be talking about.â€™ Fletcher has already been impressed by Berahino on the training ground but admitted: â€˜The lads have gone straight into him. He has said something and he will learn from it. â€˜He loves West Brom and wants to do well. Heâ€™s a young player who said something he shouldnâ€™t and he probably regrets it. Iâ€™ve done that, all young players do that. â€˜On first impressions he looks very sharp, a real goalscorer. Heâ€™s not shy!. Giving me orders straight away because he wants to score goals. Heâ€™s a nice kid welcoming, respectful and can be big influence for rest of season.â€™
"""
Test_article_tok = Test_article.split()
Test_array = np.array(Test_article_tok)
vocab = np.unique(Test_array)

test_doc_term_matrix = np.zeros((len(Test_article_tok), len(vocab)))
for i, token in enumerate(Test_article_tok):
    for j, word in enumerate(vocab):
        if word == token:
            test_doc_term_matrix[i, j] += 1

Test_doc_freq = np.sum(test_doc_term_matrix,axis=0)
Test_doc_term_freq = np.sum(test_doc_term_matrix>0,axis = 0)
idf = np.log((len(Test_article_tok)+1) / (Test_doc_term_freq + 1)) + 1
tf_test_doc = np.log(test_doc_term_matrix + 1)
tf_idf_test_doc = tf_test_doc * idf


predicted_tf_idf_sum = reg.predict(tf_idf_test_doc)


predicted_summary_tokens = []
for i in range(len(predicted_tf_idf_sum)):
    predicted_summary_tokens.append(vocab[np.argmax(predicted_tf_idf_sum[i])])

# Join the predicted summary tokens into a string
predicted_summary = " ".join(predicted_summary_tokens)

# Print the predicted summary
print(predicted_summary)


Multi-Document Text Summarization Based on Multiple Linear Regression

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Not with human summaries
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np

# Step 1: Preprocessing
# Load documents
doc1 = "First document is about machine learning."
doc2 = "Second document is about natural language processing."
doc3 = "Third document is about computer vision."
docs = [doc1, doc2, doc3]

# Tokenize documents into sentences
sentences = []
for doc in docs:
    sentences += sent_tokenize(doc)

# Step 2: Feature Extraction
# Convert sentences to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
X = X.toarray()

# Step 3: Training
# Split data into training and testing sets
num_sentences = len(sentences)
train_size = int(num_sentences * 0.8)
X_train = X[:train_size]
X_test = X[train_size:]
# Create target summary
target = ["Machine learning is important.", "Natural language processing is interesting."]
y_train = np.zeros((train_size,2))
for i in range(train_size):
    if "machine learning" in sentences[i]:
        y_train[i,0] = 1
    if "natural language processing" in sentences[i]:
        y_train[i,1] = 1
# Train linear regression model
reg1 = LinearRegression().fit(X_train, y_train[:,0])
reg2 = LinearRegression().fit(X_train, y_train[:,1])

# Step 4: Testing
# Predict summary sentences
y_pred1 = reg1.predict(X_test)
y_pred2 = reg2.predict(X_test)
y_pred = np.column_stack((y_pred1, y_pred2))
top_sentences = []
for i in np.argsort(-y_pred.sum(axis=1))[:2]:
    top_sentences.append(sentences[train_size + i])
# Generate summary
summary = " ".join(top_sentences)
print(summary)


Third document is about computer vision.


In [ ]:
# With Human summaries but also with target summary
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np

# Step 1: Preprocessing
# Load documents
doc1 = "First document is about machine learning."
doc2 = "Second document is about natural language processing."
doc3 = "Third document is about computer vision."
docs = [doc1, doc2, doc3]

# Tokenize documents into sentences
sentences = []
for doc in docs:
    sentences += sent_tokenize(doc)

# Step 2: Feature Extraction
# Convert sentences to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
X = X.toarray()

# Step 3: Training
# Create target summary
target = ["Machine learning is important.", "Natural language processing is interesting."]
y = np.zeros((len(sentences), 2))
for i in range(len(sentences)):
    if "machine learning" in sentences[i]:
        y[i, 0] = 1
    if "natural language processing" in sentences[i]:
        y[i, 1] = 1
# Load human-generated summaries
human_summaries = [
    "Machine learning is a rapidly growing field with many exciting applications.",
    "Natural language processing has the potential to revolutionize communication and information retrieval."
]
# Convert human summaries to target vectors
human_targets = np.zeros((len(human_summaries), len(sentences)))
for i, summary in enumerate(human_summaries):
    summary_sentences = sent_tokenize(summary)
    for j in range(len(sentences)):
        if sentences[j] in summary_sentences:
            human_targets[i, j] = 1

# Transpose human_targets to match the shape of y
human_targets = human_targets.T


# Train linear regression model
reg1 = LinearRegression().fit(X, human_targets[:, 0])
reg2 = LinearRegression().fit(X, human_targets[:, 1])

# Step 4: Testing
# Predict summary sentences
y_pred1 = reg1.predict(X)
y_pred2 = reg2.predict(X)
y_pred = np.column_stack((y_pred1, y_pred2))
top_sentences = []
for i in np.argsort(-y_pred.sum(axis=1))[:2]:
    top_sentences.append(sentences[i])
# Generate summary
summary = " ".join(top_sentences)
print(summary)


First document is about machine learning. Second document is about natural language processing.


In [ ]:
# Without Targeted Summary


from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [ ]:
punctuation = punctuation.replace(".", "")
punctuation += '\n'
print(punctuation)

stop_words = list(STOP_WORDS)
print(stop_words)

!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~







['about', 'other', 'these', 'full', 'still', 'whole', 'was', 'down', 'somehow', 'eleven', 'noone', 'off', 'therefore', 'our', 'every', 'yourselves', 'whenever', 'he', '’re', 'bottom', 'nowhere', 'although', 'several', 'eight', 'latter', 'your', 'hence', 'just', 'again', 'everyone', 'go', 'whether', 'elsewhere', 'under', 'otherwise', 'towards', 'would', '’ll', 'what', 'sometimes', 'her', 'fifteen', 'might', "'s", 'so', 'unless', 'whereas', 'anyway', "'m", 'between', 'be', 'seem', 'me', 'themselves', 'even', 'but', '’m', 'the', 'where', 'toward', 'by', 'whose', 'n‘t', 'however', 'thus', 'within', 'well', 'six', 'fifty', 'rather', 'of', 'cannot', 'whoever', 'anywhere', 'ourselves', 'thereby', 'how', 'because', 'two', 'therein', 'four', 'than', 'during', '‘m', 'could', 'former', 'into', 'empty', 'himself', 'or', 'when', 'take', 'been', 'neither', 'more', 'then', 'does', 'they', "'ve", 'another', 'seemed', 'becomes', 'none', 'while', 'move', 'anything'

In [ ]:
# Step 1: Preprocessing
text1 = """
A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titleyâ€™s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Rachel Titley, 28 (right). She died later from her injuries . The head-on crash took place in October 2013. Mr Eccleston-Todd's car was barely recognisable (pictured) Police said Eccleston-Todd had drunk at least three or four pints of beer before getting behind the wheel. He was found guilty of causing death by dangerous driving at Portsmouth Crown Court yesterday. Miss Titley, a 28-year-old solicitorâ€™s clerk from Cowes, Isle of Wight, had also spent the evening with friends at a pub but had not drunk any alcohol, police said. She was driving responsibly and there was â€˜nothing she could have done to avoid the collisionâ€™, they added. Lindsay Pennell, prosecuting, said: â€˜Craig Eccleston-Toddâ€™s driving resulted in the tragic death of a young woman, Rachel Titley, a death that could have been avoided. â€˜Mr Eccleston-Todd took the decision to pick up his mobile phone whilst driving and, either reading or replying to this text message, was so distracted that he failed to negotiate a left-hand bend, crossing the central white line into the path of Miss Titleyâ€™s oncoming car. Miss Titley was pulled the wreckage of herÂ Daihatsu Cuore but died later from her injuries in hospital . â€˜Miss Titley [had] a bright future ahead of her. She was also returning home having spent an enjoyable evening with friends and was driving responsibly. â€˜She had arranged to contact her friends when she got home to confirm that she had arrived safely. Her friends sadly never heard from her after they parted company. â€˜Miss Titleyâ€™s death in these circumstances reiterates the danger of using a hand-held mobile phone whilst driving.â€™ Police were unable to take breath or blood tests from Eccleston-Todd immediately, but in tests several hours after the accident he was only marginally under the drink-drive limit. The judge agreed with police that he would have been over the limit at the time his red Citroen hit Miss Titleyâ€™s blue Daihatsu Cuore on a road near Yarmouth, Isle of Wight, on October 11, 2013. His phone records showed he was also texting around the time of the crash. PC Mark Furse, from Hampshire constabularyâ€™s serious collision investigation unit, said: 'Our thoughts are with Rachel's family at this time. She had been out with friends at a pub in Shalfleet that evening, but had not had any alcohol. 'Our investigation showed that there was nothing she could have done to avoid the collision and sadly it cost her her life. 'Mr Eccleston-Todd had left work in Yarmouth and met with friends at a pub where he drank at least three to four pints of lager. He hadn't long left the pub to return home when the collision occurred at around 9.30pm. 'We weren't able to take breath or blood tests from him immediately and although blood taken several hours after the collision showed he was marginally under the limit, we maintain he would have been over the limit at the time of the collision and in summing up today, the judge agreed. 'The analysis of his phone records showed that he was texting on his phone around the time of the collision so it's highly likely this would also have contributed to his dangerous driving and loss of control.' Eccleston-Todd was found guilty of causing death by dangerous driving following a trial at Portsmouth Crown Court (pictured) He added: 'Mr Eccleston-Todd will now spend six years behind bars, but Rachel's family have lost her forever. 'I hope this will make people think twice before drinking any alcohol and getting behind the wheel, or using a phone once they're on the road. 'The dangers of drink driving and driving whilst using a mobile phone are obvious. Those who continue to do so risk spending a substantial time in prison. This case highlights just how tragic the consequences of committing these offences can be.' â€˜Mr Eccleston-Todd will now spend six years behind bars, but Rachelâ€™s family have lost her for ever. I hope this will make people think twice before drinking any alcohol and getting behind the wheel, or using a phone once theyâ€™re on the road. This case highlights just how tragic the consequences of committing these offences can be.â€™ Eccleston-Todd, of Newport, Isle of Wight, was also disqualified from driving for eight yearsÂ after which he will have to complete an extended re-test.
"""
text1 = text1.lower()

final_text = """"""
word_token_text = word_tokenize(text1)
for w in word_token_text:
  if w =='.':
    final_text += '. ' 
  elif w not in stop_words and w not in punctuation:
    final_text += ' ' + w + ' '


print(final_text)
sentences = sent_tokenize(final_text)
final_sent = []
for sent in sentences:
  if(sent != '.'):
    final_sent.append(sent)
sentences = final_sent
print(final_sent)

 drunk  driver  killed  young  woman  head-on  crash  checking  mobile  phone  jailed  years .  craig  eccleston-todd  27  driving  home  night  pub  received  text  message .  reading  replying  veered  road  driving  round  bend  smashed  rachel  titleyâ€™s  car  coming  way .  craig  eccleston-todd  27  left  mobile  phone  crashed  head-on  car  driven  rachel  titley  28  right .  died  later  injuries .  head-on  crash  took  place  october  2013.  mr  eccleston-todd  car  barely  recognisable  pictured  police  said  eccleston-todd  drunk  pints  beer  getting  wheel .  found  guilty  causing  death  dangerous  driving  portsmouth  crown  court  yesterday .  miss  titley  28-year-old  solicitorâ€™s  clerk  cowes  isle  wight  spent  evening  friends  pub  drunk  alcohol  police  said .  driving  responsibly  â€˜nothing  avoid  collisionâ€™  added .  lindsay  pennell  prosecuting  said  â€˜craig  eccleston-toddâ€™s  driving  resulted  tragic  death  young  woman  rachel  titley  

In [ ]:
# Step 2: Feature Extraction
# Convert sentences to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
X = X.toarray()

print(X)


[[0.         0.         0.         ... 0.         0.         0.28564934]
 [0.         0.         0.33643458 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.26066383 0.         0.        ]]


In [ ]:
# To do
# Make target array based on words in sentence not the whole sentenc.

# Step 3: Training
# Load human-generated summary
human_summary = """Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .
Was using phone when he veered across road in Yarmouth, Isle of Wight .
Crashed head-on into 28-year-old Rachel Titley's car, who died in hospital .
Police say he would have been over legal drink-drive limit at time of crash .
He was found guilty at Portsmouth Crown Court of causing death by dangerous driving ."""
# Convert human summary to target vector
target = np.zeros(len(sentences))
summary_sentences = sent_tokenize(human_summary)
for i in range(len(sentences)):
    if sentences[i] in summary_sentences:
      # print("Hello")
      target[i] = 1
# Train linear regression model
reg = LinearRegression().fit(X, target)


print(target)
print(len(sentences))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
33


In [ ]:
# Step 4: Testing
# Predict summary sentences
y_pred = reg.predict(X)
top_sentences = []
summ_len = int(len(sentences)*.4)
for i in np.argsort(-y_pred)[:summ_len]:
    top_sentences.append(sentences[i])
# Generate summary
summary = " ".join(top_sentences)
print(len(top_sentences))
print(top_sentences)
print(summary)


13
[' drunk  driver  killed  young  woman  head-on  crash  checking  mobile  phone  jailed  years .', 'â€˜mr  eccleston-todd  spend  years  bars  rachelâ€™s  family  lost .', 'case  highlights  tragic  consequences  committing  offences .', 'continue  risk  spending  substantial  time  prison .', "'the  dangers  drink  driving  driving  whilst  mobile  phone  obvious .", 'hope  people  think  twice  drinking  alcohol  getting  wheel  phone  road .', "eccleston-todd  found  guilty  causing  death  dangerous  driving  following  trial  portsmouth  crown  court  pictured  added  'mr  eccleston-todd  spend  years  bars  rachel  family  lost  forever .", "'the  analysis  phone  records  showed  texting  phone  time  collision  highly  likely  contributed  dangerous  driving  loss  control .", "'we  able  breath  blood  tests  immediately  blood  taken  hours  collision  showed  marginally  limit  maintain  limit  time  collision  summing  today  judge  agreed .", 'long  left  pub  return  h

Multiple Linear Regreesion without targeted Y (Not of any use)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np

# Step 1: Preprocessing
# Load documents
doc1 = "First document is about machine learning."
doc2 = "Second document is about natural language processing."
doc3 = "Third document is about computer vision."
docs = [doc1, doc2, doc3]

# Tokenize documents into sentences
sentences = []
for doc in docs:
    sentences += sent_tokenize(doc)

# Step 2: Feature Extraction
# Convert sentences to TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
X = X.toarray()

# Step 3: Training
# Load human-generated summary
human_summary = "Machine learning is a rapidly growing field with many exciting applications."
# Convert human summary to target vector
target = np.zeros(len(sentences))
summary_sentences = sent_tokenize(human_summary)
for i in range(len(sentences)):
    if sentences[i] in summary_sentences:
        target[i] = 1
# Train linear regression model
reg = LinearRegression().fit(X, target)

# Step 4: Testing
# Predict summary sentences
y_pred = reg.predict(X)
top_sentences = []
for i in np.argsort(-y_pred)[:2]:
    top_sentences.append(sentences[i])
# Generate summary
summary = " ".join(top_sentences)
print(summary)

# Multiple Linear Regression
# Load additional features
doc_lengths = np.array([len(doc) for doc in docs])
num_sentences = np.array([len(sent_tokenize(doc)) for doc in docs])
additional_features = np.column_stack((doc_lengths, num_sentences))

# Train multiple linear regression model
reg = LinearRegression().fit(np.column_stack((X, additional_features)), target)

# Predict summary sentences
X_test = vectorizer.transform(sentences).toarray()
doc_lengths_test = np.array([len(doc) for doc in docs])
num_sentences_test = np.array([len(sent_tokenize(doc)) for doc in docs])
additional_features_test = np.column_stack((doc_lengths_test, num_sentences_test))
X_test = np.column_stack((X_test, additional_features_test))

y_pred = reg.predict(X_test)
top_sentences = []
for i in np.argsort(-y_pred)[:2]:
    top_sentences.append(sentences[i])

# Generate summary
summary = " ".join(top_sentences)
print(summary)


First document is about machine learning. Second document is about natural language processing.
First document is about machine learning. Second document is about natural language processing.
